In [262]:
# Дан файл с некоторыми данными. Формат файла – произвольный, не совпадает с тем, что был в первом/втором заданиях. 
# Необходимо считать данные и добавить их к той коллекции, куда были записаны данные в первом и втором заданиях. 
# Выполните следующие запросы:
#   удалить из коллекции документы по предикату: salary < 25 000 || salary > 175000
#   увеличить возраст (age) всех документов на 1
#   поднять заработную плату на 5% для произвольно выбранных профессий
#   поднять заработную плату на 7% для произвольно выбранных городов
#   поднять заработную плату на 10% для выборки по сложному предикату 
#       (произвольный город, произвольный набор профессий, произвольный диапазон возраста)
#   удалить из коллекции записи по произвольному предикату

In [482]:
#with open('task_3_item.text', 'r', encoding='utf-8') as file:
    #content = file.read()
#lines = content.splitlines()
#for line in lines:
    #print(line)   

In [590]:
import json
from pymongo import MongoClient, collection
from bson import ObjectId

In [591]:
def read_data(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = []
    current_item = {}
    for line in lines:
        line = line.strip()
        if line.startswith("#"):
            if current_item:
                data.append(current_item)
                current_item = {}
        else:
            parts = line.split("::")
            if len(parts) == 2:
                key, value = parts
                current_item[key] = value

    if current_item:
        data.append(current_item)

    return data

In [592]:
def write_data_to_mongodb(data, host, port, db_name, collection_name):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(data)
    client.close()

In [593]:
def remove_documents_by_predicate(predicate):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    result_1 = collection.delete_many(predicate)
    return result_1

In [594]:
def increase_age():
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    non_numeric_docs = collection.find({'age': {'$not': {'$type': 'number'}}})
    collection.update_many({'age': {'$not': {'$type': 'number'}}}, {'$set': {'age': 0}})
    result_2 = collection.update_many({}, {'$inc': {'age': 1}})  # Увеличить возраст (age) всех документов на 1
    return result_2

In [595]:
def increase_salary_for_professions(professions, percent_increase):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    result_3 = collection.update_many({'job': {'$in': professions}}, {'$mul': {'salary': 1 + percent_increase / 100}})
    return result_3

In [596]:
def increase_salary_for_cities(cities, percent_increase_2):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    result_4 = collection.update_many({'city': {'$in': cities}}, {'$mul': {'salary': 1 + percent_increase_2 / 100}})
    return result_4

In [597]:
def increase_salary_complex_predicate(city_2, professions, min_age, max_age, percent_increase_3):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    result_5 = collection.update_many(
        {'$and': [
            {'city': city_2},
            {'job': {'$in': professions}},
            {'age': {'$gte': min_age, '$lte': max_age}}
        ]},
        {'$mul': {'salary': 1 + percent_increase_3 / 100}}
    )
    return result_5

In [598]:
def default_json_encoder(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

In [599]:
def write_result_to_json_del(result, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(result, file, indent=4, ensure_ascii=False, default=default_json_encoder)      

In [600]:
# Параметры
file_name = 'task_3_item.text'
host = 'localhost'
port = 27017
db_name = 'mydatabase'
collection_name = 'mycollection_3'
professions = ['Водитель', 'Учитель', 'Медсестра']
percent_increase = 5
cities = ['Краков', 'Семана', 'Хельсинки']
percent_increase_2 = 7
city_2 = ['Сория', 'Будапешт', 'Авилес']
min_age = 50
max_age = 23
percent_increase_3 = 10

In [601]:
# Чтение данных из файла
data = read_data(file_name)

In [602]:
# Запись данных в MongoDB
write_data_to_mongodb(data, host, port, db_name, collection_name)

In [603]:
# Подключение к MongoDB и получение коллекции
client = MongoClient(host, port)
db = client[db_name]
collection = db[collection_name]

In [604]:
# удалить из коллекции документы по предикату: salary < 25 000 || salary > 175000
result_1 = remove_documents_by_predicate({'$or': [{'salary': {'$lt': 25000}}, {'salary': {'$gt': 175000}}]})

In [605]:
# увеличить возраст (age) всех документов на 1
result_2 = increase_age()

In [606]:
# поднять заработную плату на 5% для произвольно выбранных профессий
result_3 = increase_salary_for_professions(professions, percent_increase)

In [607]:
# поднять заработную плату на 7% для произвольно выбранных городов
result_4 = increase_salary_for_cities(cities, percent_increase_2)

In [608]:
# поднять заработную плату на 10% для выборки по сложному предикату 
# (произвольный город, произвольный набор профессий, произвольный диапазон возраста)
result_5 = increase_salary_complex_predicate(city_2, professions, min_age, max_age, percent_increase_3)

In [609]:
# удалить из коллекции записи по произвольному предикату year > 2005
result_6 = remove_documents_by_predicate({'year': {'$gt': 2005}})

In [610]:
updated_data = list(collection.find())

In [611]:
write_result_to_json_del(updated_data, 'task_3_result.json')